# **Hosting Llama 2 with Free GPU via Google Collab**

**Before getting started, if running on Google Colab, check that the runtime is set to T4 GPU**

## Install Dependencies
- Requirements for running FastAPI Server
- Requirements for creating a public model serving URL via Ngrok
- Requirements for running Llama2 13B (including Quantization)


In [1]:
# Build Llama cpp
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.7 MB/s eta 0:00:00:00:010:01m
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 8.4 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.1.81-cp310-cp310-linux_x86_64.whl size=971718 sha256=2399bbd077ecdfd1b579908d69fb4ddfd7c1805909a442668dd2eadc816e4366
  Stored in directory: /home/nissan/.cache/pip/wheels/99/27/c0/ccda486d8dc16ebb8c61d9d7959380eced2cb3279cf0145dfe
Successfully built llama-cpp-python


In [2]:
# If this complains about dependency resolver, it's safe to ignore
!pip install fastapi[all] uvicorn python-multipart transformers pydantic tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.3 MB/s eta 0:00:00
  Obtaining dependency information for ujson!=4.0.2,!=4.1.0,!=4.2.0,!=4.3.0,!=5.0.0,!=5.1.0,>=4.0.1 from https://files.pythonhosted.org/packages/78/90/bfa62616208bd5195a113c0aa4e42c9f471e69edfc48feba6a0ab494cccb/ujson-5.8.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 7.1 MB/s eta 0:00:00


In [3]:
# This downloads and sets up the Ngrok executable in the Google Colab instance
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

--2023-08-28 21:41:13--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 52.202.168.65, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  2.19MB/s    in 6.9s    

2023-08-28 21:41:21 (1.93 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


Ngrok is used to make the FastAPI server accessible via a public URL.

Users are required to make a free account and provide their auth token to use Ngrok. The free version only allows 1 local tunnel and the auth token is used to track this usage limit.

In [4]:
# https://dashboard.ngrok.com/signup
!./ngrok authtoken 2UbsyiZi4ktGCts7yq0Hpql0agQ_4oMqiX8dQoxmSALmtyUTF

Authtoken saved to configuration file: /home/nissan/.ngrok2/ngrok.yml


## Create FastAPI App
This provides an API to the Llama 2 model. The model version can be changed in the code below as desired.

For this demo we will use the 13 billion parameter version which is finetuned for instruction (chat) following.

Despite the compression, it is still a more powerful model than the 7B variant.

In [5]:
%%writefile app.py
from typing import Any

from fastapi import FastAPI
from fastapi import HTTPException
from pydantic import BaseModel
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
import tensorflow as tf


# GGML model required to fit Llama2-13B on a T4 GPU
GENERATIVE_AI_MODEL_REPO = "TheBloke/Llama-2-13B-chat-GGML"
GENERATIVE_AI_MODEL_FILE = "llama-2-13b-chat.ggmlv3.q5_1.bin"

model_path = hf_hub_download(
    repo_id=GENERATIVE_AI_MODEL_REPO,
    filename=GENERATIVE_AI_MODEL_FILE
)

llama2_model = Llama(
    model_path=model_path,
    n_gpu_layers=64,
    n_ctx=2000
)

# Test an inference
print(llama2_model(prompt="Hello ", max_tokens=1))


app = FastAPI()


# This defines the data json format expected for the endpoint, change as needed
class TextInput(BaseModel):
    inputs: str
    parameters: dict[str, Any] | None


@app.get("/")
def status_gpu_check() -> dict[str, str]:
    gpu_msg = "Available" if tf.test.is_gpu_available() else "Unavailable"
    return {
        "status": "I am ALIVE!",
        "gpu": gpu_msg
    }


@app.post("/generate/")
async def generate_text(data: TextInput) -> dict[str, str]:
    try:
        params = data.parameters or {}
        response = llama2_model(prompt=data.inputs, **params)
        model_out = response['choices'][0]['text']
        return {"generated_text": model_out}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

Writing app.py


## Start FastAPI Server
The initial run will take a long time due to having to download the model and load it onto GPU.

Note: interrupting the Google Colab runtime will send a SIGINT and stop the server.

In [6]:
# This cell finishes quickly because it just needs to start up the server
# The server will start the model download and will take a while to start up
# ~5 minutes
!uvicorn app:app --host 0.0.0.0 --port 8000 > server.log 2>&1 &

OSError: Background processes not supported.

Check the logs at server.log to see progress.

Wait until model is loaded and check with the next cell before moving on.

In [ ]:
# If you see "Failed to connect", it's because the server is still starting up
# Wait for the model to be downloaded and the server to fully start
# Check the server.log file to see the status
!curl localhost:8000

## Use Ngrok to create a public URL for the FastAPI server.
**IMPORTANT:** If you created an account via email, please verify your email or the next 2 cells won't work.

If you signed up via Google or GitHub account, you're good to go.

In [ ]:
# This starts Ngrok and creates the public URL
from IPython import get_ipython
get_ipython().system_raw('./ngrok http 8000 &')

Check the URL generated by the next cell, it should report that the FastAPI server is alive and that GPU is available.

To hit the model endpoint, simply add `/generate` to the URL

In [ ]:
# Get the Public URL
# If this doesn't work, make sure you verified your email
# Then run the previous code cell and this one again
!curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

## Shutting Down
To shut down the processes, run the following commands in a new cell:
```
!pkill uvicorn
!pkill ngrok
```